In [2]:
import pandas as pd
import numpy as np
import time

import re

import openpyxl
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [3]:
# 수집할 맥주 목록
beer_list = ['kloud', 'fitz super clear', 'Asahi super dry', 'Tsingtao', 'Heineken',
 'Kirin ichiban', 'Sapporo Premium Beer / Draft Beer', 'stella artois', 'guinness braught',
 '1664 Blanc', 'pilsner urquell', 'San Miguel', 'OB premier pilsner', 'cass fresh',
 'stout', 'dry finish', 'max hite', 'hite extra cold', 'victoria bitter', 'BINTANG pilsner',
 'krombacher weizen', 'Miller Genuine Draft', 'Hoegaarden Cherry', 'TIGER REDLER',
 "Suntory The Premium Malt's", 'REEPER B Weissbier', 'PEEPER B IPA', 'TIGER BEER',
 'TSINGTAO WHEAT BEER', 'Erdinger Weissbier', 'Carlsberg', 'Budweiser ', 'Hoegarden',
 'YEBISU', 'Paulaner Hefe-weissbier', 'Desperados', 'Peroni Nastro Azzurro',
 'Edelweiss Snowfresh', 'Heineken Dark Lager', 'Kozel Dark 10', 'Guinness original',
 'Filite', 'SEOULITE ALE', 'JEJU WIT ALE', 'Stephans Brau Philsner', 'Stephans Brau Larger',
 'Stephans Bräu Hefe-Weizen Naturtrüb', 'Bali Hai Premium Larger', 'Apostel Brau',
 'Egger Zwickl', 'Egger Marzenbier', 'Holsten Premium Beer', 'Franzisaner Hefe-Weissbier',
 'Egger Radler Grapefruit', 'Barvaria Premium', 'Barvaria 8.6', 'Lapin Kulta IV A',
 'Grolsch Premium Larger', 'Gambrinus Original', 'Leffe Brown',
 'Lowenbrau Original', 'Asahi Super dray Black ', 'Harbin Beer', "beck's",
 'Hoegaarden Rosee', 'Platinum White Ale', 'Platinum Pale Ale', 'Ambar Especial Larger',
 'Schöfferhofer Grapefruit', 'Volfas Engelman Grünberger Hefeweizen', 'Berliner Kindl Pilsener ',
 'BURGE MEESTER', 'Red Rock', 'Erdinger Dunkel', 'Warsteiner Premium Verum', "Queen's Ale Blonde Type"]

 # 데이터프레임으로 저장
beer_list = pd.DataFrame(data=beer_list, columns=['검색이름'])

In [4]:
# 데이터 프레임 생성
data = pd.DataFrame(data=[], columns=['맥주정보', '검색이름', '맥주이름'])

# chromedriver.exe 파일 경로 설정
chromedriver = 'chromedriver.exe'
# 크롤링 할 경로 설정
url = 'https://www.ratebeer.com/search?tab=beer'
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(url)
driver.set_window_size(900, 900)

In [5]:
def crawl(driver, beer, data, k):
    # 데이터 프레임 생성
    data = pd.DataFrame(data=[], columns=['맥주정보', '검색이름', '맥주이름'])

    # url open
    print('url_open... {0} 맥주 데이터를 수집합니다..'.format(beer))
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(url)
    driver.set_window_size(900, 900)
    time.sleep(1)

    # 맥주 검색
    time.sleep(2)
    element = driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div/input')
    time.sleep(2)
    element.click()
    time.sleep(2)
    element.send_keys(beer)
    time.sleep(3)

    # 상품 선택
    driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/header/div[2]/div[1]/div[2]/div/div[2]/a[1]/div/div[2]').click()

    time.sleep(3)
    beer_name = driver.find_element(By.CSS_SELECTOR, '.MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.mt-3.MuiTypography-h4').text

    error_cnt = 0

    while 1:
        try :
            # 전체 리뷰 개수 수집
            time.sleep(3)
            string = driver.find_element(By.CLASS_NAME,'MuiTypography-root.Text___StyledTypographyTypeless-bukSfn.pzIrn.text-500.colorized__WrappedComponent-hrwcZr.hwjOn.MuiTypography-h6').text

            # ,가 포함되어 있는지에 대한 정규표현식
            extract = re.compile('[0-9]*,*[0-9]+')
            str_num = extract.findall(string)
            str_num = str_num[0]

            break
        except :
            print('오류 발생.. 다시 시작합니다.')

            error_cnt += 1

            if error_cnt == 5:
                print('연속된 오류로 다음 맥주로 넘어갑니다...')
                return

    if ',' in str_num:
        str_num = str_num.split(',')
        str_num = int(str_num[0]+str_num[1])
        num = str_num
    else:
        num = int(str_num)

    # Score breakdown 클릭
    time.sleep(3)
    element = driver.find_element(By.XPATH,'//*[@id="root"]/div[2]/div[2]/div/div/div/div[2]/div[4]/div/div[2]/div[1]/div[2]')
    time.sleep(3)
    # 해당 element로 이동
    driver.execute_script("arguments[0].click();", element)

    # 수집할 Page 수를 계산합니다.
    page_num = num // 15 + 1


    for i in range(page_num):
        print(i+1, '번째 페이지입니다.')

        # 전체 맥주 정보를 통째로 수집
        time.sleep(3)
        beer_info = driver.find_elements(By.CSS_SELECTOR,'.px-4.fj-s.f-wrap')

        tmp = []

        # 수집한 것을 데이터프레임에 저장
        for i in range(len(beer_info)):
            tmp.append(beer_info[i].text)

        tmp = pd.DataFrame(data=tmp, columns=['맥주정보'])
        tmp['맥주이름'] = beer_name
        tmp['검색이름'] = beer
        data = pd.concat([data, tmp])

        # 다음 페이지로 넘어가기 
        try :
            element = driver.find_element(By.XPATH,'//button[@title="Next page"]/span[@class="MuiIconButton-label"]')
            time.sleep(3)
            driver.execute_script("arguments[0].click();", element)
        except:
            print('마지막 페이지입니다.')

    # 데이터가 중복 수집될 경우 리뷰 수 만큼만 Cut
    if num != len(data):
        data = data[:num]

    print('리뷰수 : ', num, '수집된 리뷰수 : ', len(data))

    # 데이터를 csv파일로 저장합니다.
    result = pd.merge(data, beer_list, on='검색이름', how='left')
    result.to_csv("beer_n_"+str(k)+".csv", encoding='utf-8')

    driver.quit()

    return result

In [6]:
# 맥주 리스트 개수만큼 데이터 수집
for k in range(len(beer_list)):
    result = crawl(driver, beer_list['검색이름'].iloc[k], data, k)

url_open... Heineken 맥주 데이터를 수집합니다..
1 번째 페이지입니다.
2 번째 페이지입니다.
3 번째 페이지입니다.
4 번째 페이지입니다.
5 번째 페이지입니다.
6 번째 페이지입니다.
7 번째 페이지입니다.
8 번째 페이지입니다.
9 번째 페이지입니다.
10 번째 페이지입니다.
11 번째 페이지입니다.
12 번째 페이지입니다.
13 번째 페이지입니다.
14 번째 페이지입니다.
15 번째 페이지입니다.
16 번째 페이지입니다.
17 번째 페이지입니다.
18 번째 페이지입니다.
19 번째 페이지입니다.
20 번째 페이지입니다.
21 번째 페이지입니다.
22 번째 페이지입니다.
23 번째 페이지입니다.
24 번째 페이지입니다.
25 번째 페이지입니다.
26 번째 페이지입니다.
27 번째 페이지입니다.
28 번째 페이지입니다.
29 번째 페이지입니다.
30 번째 페이지입니다.
31 번째 페이지입니다.
32 번째 페이지입니다.
33 번째 페이지입니다.
34 번째 페이지입니다.
35 번째 페이지입니다.
36 번째 페이지입니다.
37 번째 페이지입니다.
38 번째 페이지입니다.
39 번째 페이지입니다.
40 번째 페이지입니다.
41 번째 페이지입니다.
42 번째 페이지입니다.
43 번째 페이지입니다.
44 번째 페이지입니다.
45 번째 페이지입니다.
46 번째 페이지입니다.
47 번째 페이지입니다.
48 번째 페이지입니다.
49 번째 페이지입니다.
50 번째 페이지입니다.
51 번째 페이지입니다.
52 번째 페이지입니다.
53 번째 페이지입니다.
54 번째 페이지입니다.
55 번째 페이지입니다.
56 번째 페이지입니다.
57 번째 페이지입니다.
58 번째 페이지입니다.
59 번째 페이지입니다.
60 번째 페이지입니다.
61 번째 페이지입니다.
62 번째 페이지입니다.
63 번째 페이지입니다.
64 번째 페이지입니다.
65 번째 페이지입니다.
66 번째 페이지입니다.
67 번째 페이지입니다.
68 번째 페이지입니다.
69 번째 페이지입니다.
70 번째 